## Search to find the best numbers for n_neighbors and n_components


#### This procedure was performed with the first 1000 frames of the AK peptide. The evaluation metric for the "goodness" of a reconstruction was the mean RMSD error between each frames.

#### For xyz approach with alignment:

In [1]:
import time
import mdtraj as md
import numpy as np
import scipy
from scipy import ndimage
from scipy import misc
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
import reconstruct_video as rv
import networkx as nx
import evaluate_traj as ev

In [11]:
# Read Data

# read data
t = md.load('.\\Data\\akpeptide\\ak19_2500ps.dcd', top='.\\Data\\akpeptide\\ak19.pdb')
t = t[0:1000]
t.superpose(t)

# Get trajectory info. reshape to prep for isomap
num_frames, num_atoms, num_axis = t.xyz.shape
X = np.reshape(t.xyz, (num_frames, num_atoms*num_axis))

# Shuffle the frames!!!!!!
np.random.shuffle(X)

# Save shuffled data (starting dataset)
shuffled_traj = t
shuffled_traj.xyz = np.reshape(X, (num_frames, num_atoms, num_axis))
shuffled_traj.save_dcd('optimize_aligned_xyz_shuffled.dcd')

In [ ]:
start_time = time.time()
reload(rv)
possible_n_neighbors = range(25, 45)
possible_n_components = range(20, 41)

for n_neighbors in possible_n_neighbors:
    print 'n_neighbors:', n_neighbors
    for n_components in possible_n_components:
        X_iso = manifold.Isomap(n_neighbors, n_components).fit_transform(X)
        minG = rv.get_minimum_spanning_tree(X_iso)
        order = rv.get_path_from_MSP(minG)
        reconstructed_traj = rv.get_reconstructed_traj(shuffled_traj, order)
        error = ev.get_RMSD_error(reconstructed_traj)
        print error
        
        

n_neighbors: 25


In [8]:
shuffled_traj.xyz.shape

(99L, 229L, 3L)

In [5]:
t.xyz.shape

(100L, 229L, 3L)

In [6]:
X.shape

(100L, 687L)